## Now reload and inference

In [1]:
from torch.utils.data import DataLoader

from load_dataset import load_dataset

infer_dataset = load_dataset(
    "cxr8",
    input_size=(
        1,
        448,
        448,
    ),
    clahe_tile_size=8,
)


In [2]:
print(f"{infer_dataset.imgs[0][0]} {infer_dataset[0][0].shape}")


D:\data\cxr8\filtered\no finding\00000002_000.png torch.Size([1, 448, 448])


In [ ]:
model.load_state_dict(torch.load("clahe.zip"))


In [ ]:
infer_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

model = model.cpu()
model.eval()
batch_iter = iter(infer_loader)

batch, _ = next(batch_iter)
# batch = batch.cpu()
print(batch.shape)


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from IPython.display import clear_output, display

batch, _ = next(batch_iter)
print(batch.shape)

# batch = batch.cpu()
recon_batch, mu, log_var = model(batch)
print(recon_batch.shape)

# orig_data = batch.clone().cpu().detach().numpy()
# recon_data = recon_batch.cpu().detach().numpy()

fig, ax = plt.subplots(2, 5, figsize=(20,8))

# print(v.shape)
for n in range(5):
    ax[0][n].imshow(torch.movedim(batch[n].clone().detach(), 0, -1), cmap='bone')
    ax[1][n].imshow(torch.movedim(recon_batch[n].clone().detach(), 0, -1), cmap='bone')

# display(plt.gcf())

'YYYY'